In [1]:
# install packages
!pip install numpy pandas pyarrow scikit-learn fastcore kagglehub[pandas-datasets] jinja2 boto


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from fastcore.basics import *
from fastcore.parallel import *
from os import cpu_count
import numpy as np

In [5]:
drop_columns = [
    "id",
    "Flow ID",        
    "Source IP", "Src IP",
    "Source Port", "Src Port",
    "Destination IP", "Dst IP",
    "Destination Port", "Dst Port",
    "Timestamp",
    "Attempted Category",
]

def xs_y(df, targ): 
    if not isinstance(targ, list):
        xs = df[df.columns.difference([targ])].copy()
    else:
        xs = df[df.columns.difference(targ)].copy()
    y = df[targ].copy()
    return xs, y


def get_concap_dataset(file):
    df = kagglehub.load_dataset(
          KaggleDatasetAdapter.PANDAS,
          "jozefjankaj/thesis-files",
          "concap/" + file)
    # clean the dataset
    df.drop(columns=drop_columns, inplace=True, errors='ignore')
    df.drop(columns=["category", "subcategory", "label"], inplace=True, errors='ignore')
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df["Label"] = 1

    return df

def get_cic_2018_dataset(file):
    df = kagglehub.load_dataset(
          KaggleDatasetAdapter.PANDAS,
          "jozefjankaj/thesis-files",
          "cic_2018/" + file)
    df.drop(columns=drop_columns, inplace=True, errors='ignore')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    return df

def get_cic_2017_dataset(file):
    df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "jozefjankaj/thesis-files",
        "cic/" + file)
    df.drop(columns=drop_columns, inplace=True, errors='ignore')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    return df


def train_verify_one_feature(train_df: pd.DataFrame, test_df: pd.DataFrame, feature: str):
    root = DecisionTreeClassifier(max_depth=1, criterion='gini')
    train_x, train_y = xs_y(train_df, "Label")
    test_x, test_y = xs_y(test_df, "Label")
    root.fit(train_x[feature].array.reshape(-1,1), train_y)

    predictions = root.predict(test_x[feature].array.reshape(-1,1))
    return feature, roc_auc_score(test_y, predictions)

def concap_cic_experiment(benign_df: pd.DataFrame, cic: pd.DataFrame, concap: pd.DataFrame):
    # Train on CIC => Predict ConCap
    benign_balance = benign.sample(n=cic.shape[0])
    training_df = pd.concat([benign_balance, cic]).sample(frac=1).reset_index(drop=True)
    testing_df = pd.concat([benign_balance, concap]).sample(frac=1).reset_index(drop=True)

    train_x, train_y = xs_y(training_df, "Label")
    test_x, test_y = xs_y(testing_df, "Label")

    measurements_cic_concap = pd.DataFrame([train_verify_one_feature(training_df, testing_df, feature) for feature in training_df.columns if feature != "Label"], 
                                           columns=["Feature", "ROC AUC Score"]
                                          )

    # Train on ConCap => Predict CIC
    benign_balance = benign.sample(n=cic.shape[0])
    training_df = pd.concat([benign_balance, concap]).sample(frac=1).reset_index(drop=True)
    testing_df = pd.concat([benign_balance, cic]).sample(frac=1).reset_index(drop=True)
    
    train_x, train_y = xs_y(training_df, "Label")
    test_x, test_y = xs_y(testing_df, "Label")

    measurements_concap_cic = pd.DataFrame([train_verify_one_feature(training_df, testing_df, feature) for feature in training_df.columns if feature != "Label"],
                                          columns=["Feature", "ROC AUC Score"]
                                          )
    return measurements_cic_concap, measurements_concap_cic

# SSH Bruteforce

In [3]:
bruteforce = get_cic_2018_dataset("Wednesday-14-02-2018.csv")

/tmp/ipykernel_6696/1112435009.py:36: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [4]:
bruteforce

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,115320241,17,15,901,1942,445,0,53.000000,147.544061,...,5344243,10175,9.987594e+06,5.816100e+04,10005167,9812232,-1,-1,115320241,BENIGN
1,6,116784235,27,21,3498,6304,436,0,129.555556,202.635773,...,8385364,11242,9.838241e+06,4.771012e+05,10004111,8408741,-1,-1,116784235,BENIGN
2,6,115867108,23,19,2626,4569,436,0,114.173913,195.490677,...,7780718,11225,9.808635e+06,5.990057e+05,10004152,8004832,-1,-1,115867108,BENIGN
3,6,119894757,17,16,597,768,293,0,35.117647,97.064336,...,115967,32245,9.944064e+06,2.342149e+05,10046602,9203252,-1,-1,119894757,BENIGN
4,6,112960508,57,90,9101,27113,1047,0,159.666667,274.657666,...,4316517,235349,2.694478e+07,2.276214e+07,58084562,9758551,-1,-1,112960508,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5898301,17,655,1,1,300,328,300,300,300.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,BENIGN
5898306,6,3,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,3,BENIGN
5898328,17,720,1,1,300,328,300,300,300.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,BENIGN
5898335,6,358224,5,6,2752,1408,1168,0,550.400000,547.120462,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,28135919901,BENIGN


In [5]:
benign = bruteforce[bruteforce["Label"] == "BENIGN"].copy()
benign["Label"] = 0
benign

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,115320241,17,15,901,1942,445,0,53.000000,147.544061,...,5344243,10175,9.987594e+06,5.816100e+04,10005167,9812232,-1,-1,115320241,0
1,6,116784235,27,21,3498,6304,436,0,129.555556,202.635773,...,8385364,11242,9.838241e+06,4.771012e+05,10004111,8408741,-1,-1,116784235,0
2,6,115867108,23,19,2626,4569,436,0,114.173913,195.490677,...,7780718,11225,9.808635e+06,5.990057e+05,10004152,8004832,-1,-1,115867108,0
3,6,119894757,17,16,597,768,293,0,35.117647,97.064336,...,115967,32245,9.944064e+06,2.342149e+05,10046602,9203252,-1,-1,119894757,0
4,6,112960508,57,90,9101,27113,1047,0,159.666667,274.657666,...,4316517,235349,2.694478e+07,2.276214e+07,58084562,9758551,-1,-1,112960508,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5898301,17,655,1,1,300,328,300,300,300.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,0
5898306,6,3,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,3,0
5898328,17,720,1,1,300,328,300,300,300.000000,0.000000,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,0
5898335,6,358224,5,6,2752,1408,1168,0,550.400000,547.120462,...,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,28135919901,0


In [6]:
bruteforce["Label"].unique()

array(['BENIGN', 'FTP-BruteForce - Attempted', 'SSH-BruteForce'],
      dtype=object)

In [7]:
cic_ssh = bruteforce[bruteforce["Label"] == "SSH-BruteForce"].copy()
cic_ssh["Label"] = 1
cic_ssh

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
5759575,6,364702,23,22,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,364702,1
5759577,6,393085,23,21,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,393085,1
5759582,6,397310,23,23,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,397310,1
5759583,6,372143,23,23,1944,2665,640,0,84.521739,135.953491,...,0,0,0.0,0.0,0,0,-1,-1,372143,1
5759584,6,369389,22,23,1928,2665,640,0,87.636364,137.780552,...,0,0,0.0,0.0,0,0,-1,-1,369389,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5894491,6,379886,23,21,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,379886,1
5894492,6,388358,23,23,1928,2665,640,0,83.826087,135.847392,...,0,0,0.0,0.0,0,0,-1,-1,388358,1
5894493,6,374336,25,21,1912,2665,640,0,76.480000,131.981413,...,0,0,0.0,0.0,0,0,-1,-1,374336,1
5894494,6,371640,25,21,1912,2665,640,0,76.480000,131.981413,...,0,0,0.0,0.0,0,0,-1,-1,371640,1


In [8]:
concap_ssh = get_concap_dataset("concap_ssh.csv")
concap_ssh

/tmp/ipykernel_6696/1112435009.py:22: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,4660,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0,0,0,0,-1,-1,4660,1
1,6,3557,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0,0,0,0,-1,-1,3557,1
2,6,19511,4,3,24.0,0.0,24.0,0.0,6.000000,12.000000,...,0,0,0,0,0,0,-1,-1,19511,1
3,6,11027649,22,34,1928.0,2746.0,640.0,0.0,87.636364,137.780552,...,0,0,0,0,0,0,-1,-1,11027649,1
4,6,9870,5,4,24.0,0.0,24.0,0.0,4.800000,10.733126,...,0,0,0,0,0,0,-1,-1,9870,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7633,6,4487608,17,17,1272.0,2154.0,640.0,0.0,74.823529,161.218887,...,0,0,0,0,0,0,-1,-1,4487608,1
7634,6,3367832,14,16,1272.0,2154.0,640.0,0.0,90.857143,174.415444,...,0,0,0,0,0,0,-1,-1,3367832,1
7635,6,4025378,17,17,1272.0,2154.0,640.0,0.0,74.823529,161.218887,...,0,0,0,0,0,0,-1,-1,4025378,1
7636,6,3951,4,4,24.0,0.0,24.0,0.0,6.000000,12.000000,...,0,0,0,0,0,0,-1,-1,3951,1


In [9]:
cic_concap, concap_cic = concap_cic_experiment(benign, cic_ssh, concap_ssh)

In [10]:
cic_concap.sort_values("ROC AUC Score", ascending=False).head(10)

,Feature,ROC AUC Score
70,Fwd Seg Size Min,0.994674
67,FWD Init Win Bytes,0.987924
24,Fwd IAT Min,0.783406
29,Bwd IAT Min,0.735475
12,Bwd Packet Length Mean,0.723388
56,Bwd Segment Size Avg,0.723388
66,Subflow Bwd Bytes,0.697357
46,SYN Flag Count,0.692856
16,Flow IAT Mean,0.674432
45,FIN Flag Count,0.667415


In [11]:
concap_cic.sort_values("ROC AUC Score", ascending=False).head(10)

,Feature,ROC AUC Score
30,Fwd PSH Flags,0.996242
70,Fwd Seg Size Min,0.995170
48,PSH Flag Count,0.995090
31,Bwd PSH Flags,0.992988
69,Fwd Act Data Pkts,0.991268
36,Fwd Header Length,0.986035
2,Total Fwd Packet,0.500000
4,Total Length of Fwd Packet,0.500000
9,Fwd Packet Length Std,0.500000
10,Bwd Packet Length Max,0.500000


# CIC-IDS-2017 + ConCap vs CSE-CIC-2018

In [6]:
benign = get_cic_2017_dataset("monday.csv")
benign["Label"] = 0
benign

/tmp/ipykernel_7154/2370983865.py:48: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,0,119719148,231,0,0,0,0,0,0.000000,0.000000,...,22509459,17,12685486.0,5.296658e+06,20694308,6499982,-1,-1,0,0
1,17,65511209,6,6,288,288,48,48,48.000000,0.000000,...,1506210,1506210,64004884.0,0.000000e+00,64004884,64004884,-1,-1,0,0
2,17,113976922,267,0,20447,0,153,37,76.580524,44.140625,...,10983883,14,25498178.0,1.883305e+07,48523116,5463561,-1,-1,0,0
3,17,67037196,8,8,384,384,48,48,48.000000,0.000000,...,11034681,11034681,55956316.0,0.000000e+00,55956316,55956316,-1,-1,0,0
4,17,68045057,8,8,384,384,48,48,48.000000,0.000000,...,11043596,11043596,56943904.0,0.000000e+00,56943904,56943904,-1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371618,6,5571687,4,2,0,187,0,0,0.000000,0.000000,...,108511,108511,5354085.0,0.000000e+00,5354085,5354085,-1,-1,5571687,0
371619,6,63255945,8,5,169,123,46,0,21.125000,23.111144,...,223699,223699,58780167.0,0.000000e+00,58780167,58780167,-1,-1,183308410,0
371620,17,163,2,2,104,220,52,52,52.000000,0.000000,...,0,0,0.0,0.000000e+00,0,0,-1,-1,0,0
371621,6,4476954,8,9,577,4039,342,0,72.125000,136.576967,...,0,0,0.0,0.000000e+00,0,0,-1,-1,4476954,0


In [7]:
tuesday = get_cic_2017_dataset("tuesday.csv")

/tmp/ipykernel_7154/2370983865.py:48: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [8]:
tuesday["Label"].unique()

array(['BENIGN', 'FTP-Patator - Attempted', 'FTP-Patator', 'SSH-Patator',
       'SSH-Patator - Attempted'], dtype=object)

In [9]:
cic_2017_ssh = tuesday[tuesday["Label"] == "SSH-Patator"].copy()
cic_2017_ssh["Label"] = 1
cic_2017_ssh

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
79299,6,4755497,14,15,1304,2153,640,0,93.142857,174.584719,...,0,0,0.0,0.0,0,0,-1,-1,4755497,1
79300,6,4742052,14,15,1304,2153,640,0,93.142857,174.584719,...,0,0,0.0,0.0,0,0,-1,-1,4742052,1
79301,6,1688206,10,10,1128,2009,640,0,112.800000,203.802083,...,0,0,0.0,0.0,0,0,-1,-1,1688206,1
79302,6,1884425,13,11,1128,2009,640,0,86.769231,183.298551,...,0,0,0.0,0.0,0,0,-1,-1,1884425,1
79303,6,1937542,13,11,1128,2009,640,0,86.769231,183.298551,...,0,0,0.0,0.0,0,0,-1,-1,1937542,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321777,6,11725520,22,33,2024,2745,640,0,92.000000,138.338439,...,0,0,0.0,0.0,0,0,-1,-1,11725520,1
321784,6,11641016,22,33,2024,2745,640,0,92.000000,138.338439,...,0,0,0.0,0.0,0,0,-1,-1,11641016,1
321785,6,12117686,22,33,2008,2745,640,0,91.272727,138.182137,...,0,0,0.0,0.0,0,0,-1,-1,12117686,1
321944,6,13640748,22,33,2008,2745,640,0,91.272727,138.182137,...,0,0,0.0,0.0,0,0,-1,-1,13640748,1


In [10]:
concap_ssh = get_concap_dataset("concap_ssh.csv")
concap_ssh

/tmp/ipykernel_7154/2370983865.py:22: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,4660,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0,0,0,0,-1,-1,4660,1
1,6,3557,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0,0,0,0,-1,-1,3557,1
2,6,19511,4,3,24.0,0.0,24.0,0.0,6.000000,12.000000,...,0,0,0,0,0,0,-1,-1,19511,1
3,6,11027649,22,34,1928.0,2746.0,640.0,0.0,87.636364,137.780552,...,0,0,0,0,0,0,-1,-1,11027649,1
4,6,9870,5,4,24.0,0.0,24.0,0.0,4.800000,10.733126,...,0,0,0,0,0,0,-1,-1,9870,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7633,6,4487608,17,17,1272.0,2154.0,640.0,0.0,74.823529,161.218887,...,0,0,0,0,0,0,-1,-1,4487608,1
7634,6,3367832,14,16,1272.0,2154.0,640.0,0.0,90.857143,174.415444,...,0,0,0,0,0,0,-1,-1,3367832,1
7635,6,4025378,17,17,1272.0,2154.0,640.0,0.0,74.823529,161.218887,...,0,0,0,0,0,0,-1,-1,4025378,1
7636,6,3951,4,4,24.0,0.0,24.0,0.0,6.000000,12.000000,...,0,0,0,0,0,0,-1,-1,3951,1


Put the two datasets together and mix them

In [11]:
concap_cic_2017 = pd.concat([cic_2017_ssh, concap_ssh]).sample(frac=1).reset_index(drop=True)
concap_cic_2017

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,12676227,23,34,1912.0,2746.0,640.0,0.0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,12676227,1
1,6,2252,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0.0,0.0,0,0,-1,-1,2252,1
2,6,18814,5,4,24.0,0.0,24.0,0.0,4.800000,10.733126,...,0,0,0.0,0.0,0,0,-1,-1,18814,1
3,6,15718,4,3,24.0,0.0,24.0,0.0,6.000000,12.000000,...,0,0,0.0,0.0,0,0,-1,-1,15718,1
4,6,11774056,22,33,2008.0,2745.0,640.0,0.0,91.272727,138.182137,...,0,0,0.0,0.0,0,0,-1,-1,11774056,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10594,6,2633,3,4,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0.0,0.0,0,0,-1,-1,2633,1
10595,6,2363,3,4,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0.0,0.0,0,0,-1,-1,2363,1
10596,6,5173,3,3,24.0,0.0,24.0,0.0,8.000000,13.856406,...,0,0,0.0,0.0,0,0,-1,-1,5173,1
10597,6,11595415,23,34,2008.0,2745.0,640.0,0.0,87.304348,136.339961,...,0,0,0.0,0.0,0,0,-1,-1,11595415,1


Grab the CIC 2018 SSH attack

In [12]:
# cic_2018 = get_cic_2018_dataset("Wednesday-14-02-2018.csv")
# cic_2018_ssh = cic_2018[cic_2018["Label"] == "SSH-BruteForce"].copy()
cic_2018_ssh = get_cic_2018_dataset("cic_2018_ssh.csv")
cic_2018_ssh.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
cic_2018_ssh["Label"] = 1
cic_2018_ssh

/tmp/ipykernel_7154/2370983865.py:36: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label
0,6,364702,23,22,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,364702,1
1,6,393085,23,21,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,393085,1
2,6,397310,23,23,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,397310,1
3,6,372143,23,23,1944,2665,640,0,84.521739,135.953491,...,0,0,0.0,0.0,0,0,-1,-1,372143,1
4,6,369389,22,23,1928,2665,640,0,87.636364,137.780552,...,0,0,0.0,0.0,0,0,-1,-1,369389,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94192,6,379886,23,21,1912,2665,640,0,83.130435,135.737482,...,0,0,0.0,0.0,0,0,-1,-1,379886,1
94193,6,388358,23,23,1928,2665,640,0,83.826087,135.847392,...,0,0,0.0,0.0,0,0,-1,-1,388358,1
94194,6,374336,25,21,1912,2665,640,0,76.480000,131.981413,...,0,0,0.0,0.0,0,0,-1,-1,374336,1
94195,6,371640,25,21,1912,2665,640,0,76.480000,131.981413,...,0,0,0.0,0.0,0,0,-1,-1,371640,1


In [16]:
cic_2018_ssh.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd RST Flags',
       'Bwd RST Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min',
       'Packet Length Max', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', '

In [13]:
result_separate, _ = concap_cic_experiment(benign, cic_2017_ssh, cic_2018_ssh)

In [14]:
result_separate.sort_values("ROC AUC Score", ascending=False).head(10)

,Feature,ROC AUC Score
30,Fwd PSH Flags,0.994259
48,PSH Flag Count,0.992908
69,Fwd Act Data Pkts,0.991895
31,Bwd PSH Flags,0.988855
36,Fwd Header Length,0.970449
2,Total Fwd Packet,0.959473
6,Fwd Packet Length Max,0.937015
70,Fwd Seg Size Min,0.921648
8,Fwd Packet Length Mean,0.890365
55,Fwd Segment Size Avg,0.890365


In [20]:
result_together, _ = concap_cic_experiment(benign, concap_cic_2017, cic_2018_ssh)

In [21]:
result_together.sort_values("ROC AUC Score", ascending=False).head(10)

,Feature,ROC AUC Score
69,Fwd Act Data Pkts,0.990235
31,Bwd PSH Flags,0.988584
4,Total Length of Fwd Packet,0.965563
6,Fwd Packet Length Max,0.944051
70,Fwd Seg Size Min,0.923200
67,FWD Init Win Bytes,0.891358
24,Fwd IAT Min,0.885499
37,Bwd Header Length,0.815596
45,FIN Flag Count,0.814935
46,SYN Flag Count,0.814228


# Testing

In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "dhoogla/nfuqnids",
  "NF-UQ-NIDS.parquet",
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)


df

/home/messik/School/AJ24-25/Thesis/code/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_7154/1832202039.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78.9M/78.9M [00:11<00:00, 6.95MB/s]

Extracting zip of NF-UQ-NIDS.parquet...


,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack,Dataset
0,62073,56082,6,0.000000,9672,416,11,8,25,15,0,Benign,NF-UNSW-NB15
1,32284,1526,6,0.000000,1776,104,6,2,25,0,0,Benign,NF-UNSW-NB15
2,21,21971,6,1.000000,1842,1236,26,22,25,1111,0,Benign,NF-UNSW-NB15
3,23800,46893,6,0.000000,528,8824,10,12,27,124,0,Benign,NF-UNSW-NB15
4,63062,21,6,1.000000,1786,2340,32,34,25,1459,0,Benign,NF-UNSW-NB15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9156320,80,80,6,7.000000,2330065,0,2523,0,0,4263037,0,Benign,NF-BoT-IoT
9156321,0,0,6,0.000000,1054423,0,1513,0,0,4263062,0,Benign,NF-BoT-IoT
9156322,365,565,17,0.000000,62422,0,1357,0,0,4263062,0,Benign,NF-BoT-IoT
9156323,50850,8883,6,222.177994,11300,1664,32,32,24,4264935,0,Benign,NF-BoT-IoT


In [2]:
df["Attack"].unique()

array(['Benign', 'Exploits', 'Reconnaissance', 'DoS', 'Generic',
       'Shellcode', 'Backdoor', 'Fuzzers', 'Worms', 'Analysis',
       'injection', 'DDoS', 'scanning', 'password', 'mitm', 'xss',
       'ransomware', 'Infilteration', 'Bot', 'Brute Force', 'Theft'],
      dtype=object)